# Analyzing Business Data in SQL (SQL queries examples)

<img src="https://info-comp.ru/wp-content/uploads/images/stories/kartinki2/What_is_SQL_1.jpg">

<p>Businesses track data on everything, from operations to marketing to HR. Leveraging this data enables businesses to better understand themselves and their customers, leading to higher profits and better performance. In this notebook we will review the key metrics that businesses use to measure performance, write SQL queries to calculate these metrics and produce report-ready results.</p>

## 1. Revenue, cost, and profit

<p>Profit is one of the first things people use to assess a company's success. In this paragraph, will'll discover how to calculate revenue and cost, and then combine the two calculations using Common Table Expressions to calculate profit.</p>

<p><b><i>PROFIT:</i></b> the money company makes minus the money it spends. (<i>revenue - cost</i>)</p>
<p><b><i>REVENUE:</i></b> the money company makes.</p>
<p><b><i>COST:</i></b> the money company spends.</p>

<h3>EXAMPLE_01: Revenue per customer</h3>

<code><span style="color:green">-- Calculate revenue</span>
<span style="color:blue">SELECT sum(meals.meal_price * orders.order_quantity) AS revenue
  FROM meals
  JOIN orders ON meals.meal_id = orders.meal_id</span>
<span style="color:green">-- Keep only the records of customer ID 15</span>
<span style="color:blue">WHERE user_id = 15;</span>
</code>

<h3>EXAMPLE_02: Revenue per week</h3>

<code><span style="color:blue">SELECT DATE_TRUNC('week', order_date) :: DATE AS delivr_week,</span>
       <span style="color:green">-- Calculate revenue</span>
       <span style="color:blue">sum(orders.order_quantity * meals.meal_price) AS revenue
  FROM meals
  JOIN orders ON meals.meal_id = orders.meal_id</span>
<span style="color:green">-- Keep only the records in June 2018</span>
<span style="color:blue">WHERE DATE_TRUNC('month', order_date) = '2018-06-01'
GROUP BY delivr_week
ORDER BY delivr_week ASC;</span></code>

<h3>EXAMPLE_03: Total cost</h3>

<code><span style="color:blue">SELECT sum(meals.meal_cost * stock.stocked_quantity) AS total_cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id</span></code>

<h3>EXAMPLE_04: Top meals by cost</h3>

<code><span style="color:blue">SELECT</span>
<span style="color:green">-- Calculate cost per meal ID</span>
  <span style="color:blue">meals.meal_id,
  sum(stock.stocked_quantity * meals.meal_cost) AS cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id
GROUP BY meals.meal_id
ORDER BY cost DESC</span>
<span style="color:green">-- Only the top 5 meal IDs by purchase cost</span>
<span style="color:blue">LIMIT 5;</span></code>

<h3>EXAMPLE_05: Using CTEs</h3>

<code><span style="color:green">-- Declare a CTE named monthly_cost</span>
<span style="color:blue">WITH monthly_cost AS (
  SELECT
    DATE_TRUNC('month', stocking_date)::DATE AS delivr_month,
    SUM(meal_cost * stocked_quantity) AS cost
  FROM meals
  JOIN stock ON meals.meal_id = stock.meal_id
  GROUP BY delivr_month)</span>


<span style="color:blue">SELECT</span>
  </span><span style="color:green">-- Calculate the average monthly cost before September</span>
<span style="color:blue">  avg(cost)
FROM monthly_cost
WHERE delivr_month < '2018-09-01';</span></code>

<h3>EXAMPLE_06: Profit per eatery</h3>

<code><span style="color:blue">WITH revenue AS (<span>
  <span style="color:green">-- Calculate revenue per eatery</span>
  <span style="color:blue">SELECT eatery,
         SUM(meal_price * order_quantity) AS revenue
    FROM meals
    JOIN orders ON meals.meal_id = orders.meal_id
   GROUP BY eatery),</span>

  <span style="color:blue">cost AS (</span>
  <span style="color:green">-- Calculate cost per eatery</span>
  <span style="color:blue">SELECT eatery,
         SUM(meal_cost * stocked_quantity) AS cost
    FROM meals
    JOIN stock ON meals.meal_id = stock.meal_id
   GROUP BY eatery)</span>

   <span style="color:green">-- Calculate profit per eatery</span>
   <span style="color:blue">SELECT revenue.eatery,
          revenue - cost AS profit
     FROM revenue
     JOIN cost ON revenue.eatery = cost.eatery
    ORDER BY profit DESC;</span></code>

<h3>EXAMPLE_07: Profit per month</h3>

<code><span style="color:green">-- Set up the revenue CTE</span>
<span style="color:blue">WITH revenue AS ( 
	SELECT
		DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
		sum(meals.meal_price * orders.order_quantity) AS revenue
	FROM meals
	JOIN orders ON meals.meal_id = orders.meal_id
	GROUP BY delivr_month),</span>
<span style="color:green">-- Set up the cost CTE</span>
  <span style="color:blue">cost AS (
 	SELECT
		DATE_TRUNC('month', stocking_date) :: DATE AS delivr_month,
		sum(stock.stocked_quantity * meals.meal_cost) AS cost
	FROM meals
    JOIN stock ON meals.meal_id = stock.meal_id
	GROUP BY delivr_month)</span>


<span style="color:green">-- Calculate profit by joining the CTEs</span>
<span style="color:blue">SELECT
	revenue.delivr_month,
	(revenue - cost) AS profit
FROM revenue
JOIN cost ON revenue.delivr_month = cost.delivr_month
ORDER BY revenue.delivr_month ASC;</span></code>

## 2. User-centric KPIs

<p>Financial KPIs like profit are important, but they don't speak to user activity and engagement. In this paragraph , we'll discover how to calculate the registrations and active users KPIs, and use window functions to calculate the user growth and retention rates.</p>

<p><b><i>REGISTRATION:</i></b> when a user first signs up for an account on service through its app.</p>
<p><b><i>REGISTRATIONS KPI:</i></b> counts registrations over time, usually per month (good at measuring  a company`s success in attracting new users).</p>
<p><b><i>ACTIVE USERS KPI:</i></b> counts the active users of a company`s app over a time period 
    <ul>
        <li>by day (daily active users, or DAU)</li>
        <li>by month (monthly active users, or MAU)</li>
    </ul>  
Stickiness <b>(DAU/MAU)</b>, measures how often users engage with an app on average (If service's stickiness is DAU / MAU = 0.3 (30%), users use servises for 30% * 30 days = 9 days each month on average).
</p>
<p><b><i>GROWTH RATE:</i></b> a percentage that show the change in a variable over time relative to that variable's initial value. ($\frac{current_value - previous_value}{previous_value}$)</p>
<p><b><i>RETENTION RATE:</i></b> a percentage measuring how many users who were active in a previous month are still active in the current month. (<b>formula:</b> $\frac{Uc}{Up}$, where <i>U<sub>c</sub></i> is the count of distinct users who were active in both the current and previous months, and <i>U<sub>p</sub></i> is the count of distinct users who were active in the previous period).

<h3>EXAMPLE_08: Registrations by month</h3>

<code><span style="color:green">-- Wrap the query you wrote in a CTE named reg_dates</span>
<span style="color:blue">WITH registr_dates AS (
  SELECT
    user_id,
    MIN(order_date) AS registration_date
  FROM orders
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Count the unique user IDs by registration month</span>
  <span style="color:blue">DATE_TRUNC('month', registration_date) :: DATE AS delivr_month,
  COUNT(DISTINCT user_id) AS num_user_registered
FROM registr_dates
GROUP BY delivr_month
ORDER BY delivr_month ASC;</span> 
</code>

<h3>EXAMPLE_09: Monthly active users (MAU)</h3>

<code><span style="color:blue">SELECT</span>
  <span style="color:green">-- Truncate the order date to the nearest month</span>
  <span style="color:blue">DATE_TRUNC('month', order_date) :: DATE AS delivr_month,</span>
  <span style="color:green">-- Count the unique user IDs</span>
  <span style="color:blue">COUNT(DISTINCT(user_id)) AS mau
FROM orders
GROUP BY delivr_month</span>
<span style="color:green">-- Order by month</span>
<span style="color:blue">ORDER BY delivr_month ASC;</span></code>

<h3>EXAMPLE_10: Registrations running total (Commutative Sum)</h3>

<code><span style="color:blue">WITH reg_dates AS (
  SELECT
    user_id,
    MIN(order_date) AS reg_date
  FROM orders
  GROUP BY user_id),</span>

  <span style="color:blue">regs AS (
  SELECT
    DATE_TRUNC('month', reg_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS regs
  FROM reg_dates
  GROUP BY delivr_month)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Calculate the registrations running total by month</span>
  <span style="color:blue">delivr_month,
  regs,
  SUM(regs) OVER (ORDER BY delivr_month ASC) AS regs_rt
FROM regs</span>
<span style="color:green">-- Order by month in ascending order</span>
<span style="color:blue">ORDER BY delivr_month ASC;</span> </code>

<h3>EXAMPLE_11: Lagged MAU - query</h3>

<code><span style="color:blue">WITH mau AS (
  SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS mau
  FROM orders
  GROUP BY delivr_month)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Select the month and the MAU</span>
  <span style="color:blue">delivr_month,
  mau,
  COALESCE(
    LAG(mau) OVER (ORDER BY delivr_month ASC),
  0) AS last_mau
FROM mau</span>
<span style="color:green">-- Order by month in ascending order</span>
<span style="color:blue">ORDER BY delivr_month ASC;</span></code>

<h3>EXAMPLE_12: Deltas - query</h3>

<code><span style="color:blue">WITH mau AS (
  SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS mau
  FROM orders
  GROUP BY delivr_month),</span>

  <span style="color:blue">mau_with_lag AS (
  SELECT
    delivr_month,
    mau,<span>
    <span style="color:green">-- Fetch the previous month's MAU</span>
    <span style="color:blue">COALESCE(
      lag(mau) OVER (ORDER BY delivr_month ASC),
    0) AS last_mau
  FROM mau)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Calculate each month's delta of MAUs</span>
  <span style="color:blue">delivr_month,
  (mau - last_mau) AS mau_delta
FROM mau_with_lag</span>
<span style="color:green">-- Order by month in ascending order</span>
<span style="color:blue">ORDER BY delivr_month;</span></code>

<h3>EXAMPLE_13: Growth rate</h3>

<code><span style="color:blue">WITH mau AS (
  SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS mau
  FROM orders
  GROUP BY delivr_month),</span>

  <span style="color:blue">mau_with_lag AS (
  SELECT
    delivr_month,
    mau,
    GREATEST(
      LAG(mau) OVER (ORDER BY delivr_month ASC),
    1) AS last_mau
  FROM mau)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Calculate the MoM MAU growth rates</span>
  <span style="color:blue">delivr_month,
  ROUND(
    (mau - last_mau)::NUMERIC / last_mau,
  2) AS growth
FROM mau_with_lag</span>
<span style="color:green">-- Order by month in ascending order</span>
<span style="color:blue">ORDER BY delivr_month;</span></code>

<h3>EXAMPLE_14: Order growth rate</h3>

<code><span style="color:blue">WITH orders AS (
  SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,</span>
    <span style="color:green">--  Count the unique order IDs</span>
    <span style="color:blue">COUNT(DISTINCT(order_id)) AS orders
  FROM orders
  GROUP BY delivr_month),</span>

  <span style="color:blue">orders_with_lag AS (
  SELECT
    delivr_month,</span>
    <span style="color:green">-- Fetch each month's current and previous orders</span>
    <span style="color:blue">orders,
    COALESCE(
      lag(orders) OVER (ORDER BY delivr_month),
    1) AS last_orders
  FROM orders)</span>

<span style="color:blue">SELECT
  delivr_month,</span>
  <span style="color:green">-- Calculate the MoM order growth rate</span>
  <span style="color:blue">ROUND(
    (orders-last_orders)::NUMERIC / last_orders,
  2) AS growth
FROM orders_with_lag
ORDER BY delivr_month ASC;</span></code>

<h3>EXAMPLE_15: Retention rate</h3>

<code><span style="color:blue">WITH user_monthly_activity AS (
  SELECT DISTINCT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    user_id
  FROM orders)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Calculate the MoM retention rates</span>
  <span style="color:blue">previous.delivr_month,
  ROUND(
    COUNT(DISTINCT current.user_id) :: NUMERIC /GREATEST(COUNT(DISTINCT previous.user_id), 1),
  2) AS retention_rate
FROM user_monthly_activity AS previous
LEFT JOIN user_monthly_activity AS current</span>
<span style="color:green">-- Fill in the user and month join conditions</span>
<span style="color:blue">ON  previous.user_id = current.user_id
AND  previous.delivr_month = (current.delivr_month - INTERVAL '1 month')
GROUP BY previous.delivr_month
ORDER BY previous.delivr_month ASC;</span></code>

## 3. ARPU, histograms, and percentiles

<p>Since a KPI is a single number, it can't describe how data is distributed. In this paragraph, we'll discover about unit economics, histograms, bucketing, and percentiles, which we can use to spot the variance in user behaviors.</p>

<p><b><i>UNIT ECONOMICS:</i></b> measures perfomance per unit, as opposed to overall perfomance.</p>
<ul>
    <li>AVERAGE REVENUE PER USER (<b>ARPU</b>)</li>
    <ul><li>$\frac{Revenue}{Count-of-users}$</li></ul>
</ul>
<p><b><i>Percentile:</i></b>n<sub>th</sub> percentile is the value for which n% of dataset's values are beneath this value.</p>

<h3>EXAMPLE_16: Average revenue per user</h3>

<code><span style="color:green">-- Create a CTE named kpi</span>
<span style="color:blue">WITH kpi AS (
  SELECT
    <span style="color:green">-- Select the user ID and calculate revenue</span>
    user_id,
    SUM(m.meal_price * o.order_quantity) AS revenue
  FROM meals AS m
  JOIN orders AS o ON m.meal_id = o.meal_id
  GROUP BY user_id)</span>
<span style="color:green">-- Calculate ARPU</span>
<span style="color:blue">SELECT ROUND(AVG(revenue) :: NUMERIC,2) AS arpu
FROM kpi;</span></code>

<h3>EXAMPLE_17: ARPU per week</h3>

<code><span style="color:blue">WITH kpi AS (
  SELECT</span>
    <span style="color:green">-- Select the week, revenue, and count of users</span>
    <span style="color:blue">DATE_TRUNC('week', order_date) :: DATE AS delivr_week,
    SUM(order_quantity*meal_price) AS revenue,
    COUNT(DISTINCT(user_id)) AS users
  FROM meals AS m
  JOIN orders AS o ON m.meal_id = o.meal_id
  GROUP BY delivr_week)</span>

<span style="color:blue">SELECT
  delivr_week,</span>
  <span style="color:green">-- Calculate ARPU</span>
  <span style="color:blue">ROUND(
    revenue :: NUMERIC / users,
  2) AS arpu
FROM kpi</span>
<span style="color:green">-- Order by week in ascending order</span>
<span style="color:blue">ORDER BY delivr_week ASC;</span></code>

<h3>EXAMPLE_18: Average orders per user</h3>

<code><span style="color:blue">WITH kpi AS (
  SELECT</span>
    <span style="color:green">-- Select the count of orders and users</span>
    <span style="color:blue">COUNT(DISTINCT order_id) AS orders,
    COUNT(DISTINCT user_id) AS users
  FROM orders)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Calculate the average orders per user</span>
  <span style="color:blue">ROUND(
    orders :: NUMERIC / GREATEST(users, 1),
  2) AS arpu
FROM kpi;</span></code>

<h3>EXAMPLE_19: ARPU</h3>

<code><span style="color:blue">WITH kpis AS (
    SELECT 
        SUM(meal_price * order_quantity) AS revenue,
        COUNT(DISTINCT user_id) AS users 
    FROM meals JOIN orders ON meals.meal_id = orders.meal_id
    )</span>
    <span style="color:blue">SELECT
        ROUND(revenue :: NUMERIC / GREATEST(users, 1),2) AS arpu 
    FROM kpis;</span>
</code>

<h3>EXAMPLE_20: Histogram of revenue</h3>

<code><span style="color:blue">WITH user_revenues AS (
  SELECT</span>
    <span style="color:green">-- Select the user ID and revenue</span>
    <span style="color:blue">user_id,
    SUM(meal_price*order_quantity) AS revenue
  FROM meals AS m
  JOIN orders AS o ON m.meal_id = o.meal_id
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Return the frequency table of revenues by user</span>
  <span style="color:blue">ROUND(revenue::NUMERIC,-2) AS revenue_100,
  COUNT(DISTINCT(user_id)) AS users
FROM user_revenues
GROUP BY revenue_100
ORDER BY revenue_100 ASC;</span></code>

<h3>EXAMPLE_21: Histogram of orders</h3>

<code><span style="color:blue">WITH user_orders AS (
  SELECT
    user_id,
    COUNT(DISTINCT order_id) AS orders
  FROM orders
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Return the frequency table of orders by user</span>
  <span style="color:blue">orders,
  COUNT(DISTINCT user_id) AS users
FROM user_orders
GROUP BY orders
ORDER BY orders ASC;</span></code>

<h3>EXAMPLE_22: Bucketing users by revenue</h3>

<code><span style="color:blue">WITH user_revenues AS (
  SELECT</span>
    <span style="color:green">-- Select the user IDs and the revenues they generate</span>
    <span style="color:blue">user_id,
    sum(meal_price*order_quantity) AS revenue
  FROM meals AS m
  JOIN orders AS o ON m.meal_id = o.meal_id
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Fill in the bucketing conditions</span>
  <span style="color:blue">CASE
    WHEN revenue < 150 THEN 'Low-revenue users'
    WHEN revenue < 300 THEN 'Mid-revenue users'
    ELSE 'High-revenue users'
  END AS revenue_group,
  COUNT(DISTINCT(user_id)) AS users
FROM user_revenues
GROUP BY revenue_group;</span></code>

<h3>EXAMPLE_23: Bucketing users by orders</h3>

<code><span style="color:green">-- Store each user's count of orders in a CTE named user_orders</span>
<span style="color:blue">WITH user_orders AS (
  SELECT
    user_id,
    COUNT(DISTINCT(order_id)) AS orders
  FROM orders
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Write the conditions for the three buckets</span>
  <span style="color:blue">CASE
    WHEN orders < 8 THEN 'Low-orders users'
    WHEN orders < 15 THEN 'Mid-orders users'
    ELSE 'High-orders users'
  END AS order_group,</span>
  <span style="color:green">-- Count the distinct users in each bucket</span>
  <span style="color:blue">COUNT(DISTINCT(user_id)) AS users
FROM user_orders
GROUP BY order_group;</span></code>

<h3>EXAMPLE_24: Revenue quartiles</h3>

<code><span style="color:blue">WITH user_revenues AS (</span>
  <span style="color:green">-- Select the user IDs and their revenues</span>
  <span style="color:blue">SELECT
    user_id,
    sum(meal_price*order_quantity) AS revenue
  FROM meals AS m
  JOIN orders AS o ON m.meal_id = o.meal_id
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Calculate the first, second, and third quartile</span>
  <span style="color:blue">ROUND(
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY revenue ASC) :: NUMERIC,
  2) AS revenue_p25,
  ROUND(
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY revenue ASC) :: NUMERIC,
  2) AS revenue_p50,
  ROUND(
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY revenue ASC) :: NUMERIC,
  2) AS revenue_p75,</span>
  <span style="color:green">-- Calculate the average</span>
  <span style="color:blue">ROUND(avg(revenue) :: NUMERIC, 2) AS avg_revenue
FROM user_revenues;</span></code>

<h3>EXAMPLE_25: Interquartile range</h3>

<code><span style="color:blue">WITH user_revenues AS (
  SELECT</span>
    <span style="color:green">-- Select user_id and calculate revenue by user</span> 
    <span style="color:blue">user_id,
    SUM(m.meal_price * o.order_quantity) AS revenue
  FROM meals AS m
  JOIN orders AS o ON m.meal_id = o.meal_id
  GROUP BY user_id),</span>

  <span style="color:blue">quartiles AS (
  SELECT</span>
    <span style="color:green">-- Calculate the first and third revenue quartiles</span>
    <span style="color:blue">ROUND(
      PERCENTILE_CONT(0.25) WITHIN GROUP
      (ORDER BY revenue ASC) :: NUMERIC,
    2) AS revenue_p25,
    ROUND(
      PERCENTILE_CONT(0.75) WITHIN GROUP
      (ORDER BY revenue ASC) :: NUMERIC,
    2) AS revenue_p75
  FROM user_revenues)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Count the number of users in the IQR</span>
  <span style="color:blue">COUNT(DISTINCT user_id) AS users
FROM user_revenues
CROSS JOIN quartiles</span>
<span style="color:green">-- Only keep users with revenues in the IQR range</span>
<span style="color:blue">WHERE revenue :: NUMERIC >= revenue_p25
  AND revenue :: NUMERIC <= revenue_p75;</span></code>

## 4. Generating an executive report

<p>Executives often use the KPIs we've calculated in the three previous paragraphs to guide business decisions. In this paragraph, we'll package the KPIs we've created into a readable report you can present to managers and executives.</p>

<p><b><i>TO_CHAR(DATE, TEXT):</i></b> (the formatted date string). <a href="https://www.postgresql.org/docs/9.6/functions-formatting.html">Function documentation</a></p>
<p><b><i>WINDOW FUNCTIONS:</i></b>
    <ul>
        <li><b>SUM(...) OVER (...): </b>Calculates a column running total;<ul><li><i>EXAMPLE:</i> <b>SUM(registrations) OVER (ORDER BY registration_month)</b> - calculates the regestrations running total</li></ul></li>
        <li><b>LAG(...) OVER (...): </b>Fetches a precedin grow's value;<ul><li><i>EXAMPLE:</i> <b>LAG(mau) OVER (ORDER BY active_month)</b> - returns the previous month's active users(MAU)</li></ul></li>
        <li><b>RANK() OVER (...): </b>Assigns a rank to each row based on that row's position in a sorted order;<ul><li><i>EXAMPLE:</i> <b>RANK() OVER (ORDER BY revenue DESC)</b> - ranks users, eateries, or months by the revenue they've generated
</li></ul></li>
    </ul>
</p>
<p><b><i>PIVOTING: </i></b>Rotating a table around a pivot column; transposing a column's values into columns (Converts a "long" table into a "wide" one).</p>

<h3>EXAMPLE_26: Formatting dates</h3>

<code><span style="color:blue">SELECT DISTINCT</span>
  <span style="color:green">-- Select the order date</span>
  <span style="color:blue">order_date,</span>
  <span style="color:green">-- Format the order date</span>
  <span style="color:blue">TO_CHAR(order_date, 'FMDay DD, FMMonth YYYY') AS format_order_date
FROM orders
ORDER BY order_date ASC
LIMIT 3;</span></code>

<h3>EXAMPLE_27: Rank users by their count of orders</h3>

<code><span style="color:green">-- Set up the user_count_orders CTE</span>
<span style="color:blue">WITH user_count_orders AS (
  SELECT
    user_id,
    COUNT(DISTINCT order_id) AS count_orders
  FROM orders</span>
  <span style="color:green">-- Only keep orders in August 2018</span>
  <span style="color:blue">WHERE DATE_TRUNC('month', order_date) = '2018-08-01'
  GROUP BY user_id)</span>

<span style="color:blue">SELECT</span>
  <span style="color:green">-- Select user ID, and rank user ID by count_orders</span>
  <span style="color:blue">user_id,
  RANK() OVER (ORDER BY count_orders DESC) AS count_orders_rank
FROM user_count_orders
ORDER BY count_orders_rank ASC</span>
<span style="color:green">-- Limit the user IDs selected to 3</span>
<span style="color:blue">LIMIT 3;</span></code>

<h3>EXAMPLE_28: Pivoting user revenues by month</h3>

<code><span style="color:green">-- Import tablefunc</span>
<span style="color:blue">CREATE EXTENSION IF NOT EXISTS tablefunc;</span>

<span style="color:blue">SELECT * FROM CROSSTAB($$
  SELECT
    user_id,
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    SUM(meal_price * order_quantity) :: FLOAT AS revenue
  FROM meals
  JOIN orders ON meals.meal_id = orders.meal_id
 WHERE user_id IN (0, 1, 2, 3, 4)
   AND order_date < '2018-09-01'
 GROUP BY user_id, delivr_month
 ORDER BY user_id, delivr_month;
$$)</span>
<span style="color:green">-- Select user ID and the months from June to August 2018</span>
<span style="color:blue">AS ct (user_id INT,
       "2018-06-01" FLOAT,
       "2018-07-01" FLOAT,
       "2018-08-01" FLOAT)
ORDER BY user_id ASC;</span></code>

<h3>EXAMLE_29: Another pivot table example</h3>

<code><span style="color:green">-- Import tablefunc</span>
<span style="color:blue">CREATE EXTENSION IF NOT EXISTS tablefunc;</span>

<span style="color:blue">SELECT * FROM CROSSTAB($$
  SELECT
    -- Select eatery and calculate total cost
    eatery,
    DATE_TRUNC('month', stocking_date) :: DATE AS delivr_month,
    SUM(meal_cost * stocked_quantity) :: FLOAT AS cost
  FROM meals
  JOIN stock ON meals.meal_id = stock.meal_id
-- Keep only the records after October 2018
  WHERE DATE_TRUNC('month', stocking_date) > '2018-10-01'
  GROUP BY eatery, delivr_month
  ORDER BY eatery, delivr_month;
$$)</span>

<span style="color:green">-- Select the eatery and November and December 2018 as columns</span>
<span style="color:blue">AS ct (eatery TEXT,
       "2018-11-01" FLOAT,
       "2018-12-01" FLOAT)
ORDER BY eatery ASC;</span></code>

<h3>EXAMPLE_30: Executive report</h3>

<code><span style="color:green">-- Import tablefunc</span>
<span style="color:blue">CREATE EXTENSION IF NOT EXISTS tablefunc;</span>

<span style="color:green">-- Pivot the previous query by quarter</span>
<span style="color:blue">SELECT * FROM CROSSTAB($$
  WITH eatery_users AS  (
    SELECT
      eatery,
 -- Format the order date so "2018-06-01" becomes "Q2 2018"
      TO_CHAR(order_date, '"Q"Q YYYY') AS delivr_quarter,
 -- Count unique users
      COUNT(DISTINCT user_id) AS users
    FROM meals
    JOIN orders ON meals.meal_id = orders.meal_id
    GROUP BY eatery, delivr_quarter
    ORDER BY delivr_quarter, users)
 --
  SELECT
 -- Select eatery and quarter
    eatery,
    delivr_quarter,
 -- Rank rows, partition by quarter and order by users
    RANK() OVER
      (PARTITION BY delivr_quarter
       ORDER BY users DESC) :: INT AS users_rank
  FROM eatery_users
  ORDER BY eatery, delivr_quarter;
$$)</span>
<span style="color:green">-- Select the columns of the pivoted table</span>
<span style="color:blue">AS  ct (eatery TEXT,
        "Q2 2018" INT,
        "Q3 2018" INT,
        "Q4 2018" INT)
ORDER BY "Q4 2018";</span></code>